<a href="https://colab.research.google.com/github/siriusted/gym-dssat-notebooks/blob/master/SB3_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gym-DSSAT x Stable-Baselines3 Tutorial

Welcome to a brief introduction to using gym-dssat with stable-baselines3.

For a background or more details about using stable-baselines3 for reinforcement learning, please take a look [here](https://github.com/araffin/rl-tutorial-jnrr19/tree/sb3)

In this notebook, we will assume familiarity with reinforcement learning and stable-baselines3. Thus the focus is on interacting with gym-dssat using SB3.

Next we proceed with installations

# Installation
- gym_dssat

In [ ]:
!echo "deb [ arch=amd64 ] https://raw.githubusercontent.com/pdidev/repo/ubuntu bionic main" | sudo tee /etc/apt/sources.list.d/pdi.list > /dev/null
!wget -O /etc/apt/trusted.gpg.d/pdidev-archive-keyring.gpg https://raw.githubusercontent.com/pdidev/repo/ubuntu/pdidev-archive-keyring.gpg
!chmod a+r /etc/apt/trusted.gpg.d/pdidev-archive-keyring.gpg /etc/apt/sources.list.d/pdi.list
!apt update &> /dev/null
!apt install pdidev-archive-keyring libpdi-dev &> /dev/null

!wget  http://gac.udc.es/~emilioj/bionic.tgz
!tar -xf bionic.tgz
!cd bionic/ && apt install `find . -name "*.deb"` &> /dev/null

!pip install -U PyYAML &> /dev/null

# add newly installed libraries to path
import sys
sys.path.append('/opt/gym_dssat_pdi/lib/python3.7/site-packages')

- stable_baselines3

In [ ]:
!pip install stable-baselines3[extra] &> /dev/null

All set! 

Next, we will train a PPO agent using stable-baselines3. This agent will be compared to two hardcoded agents, namely a Null agent and an Expert agent.

We will use a wrapper below to make for easy interaction with SB3, as well as plotting utilities.

In [ ]:
import gym
import gym_dssat_pdi
import numpy as np

# helpers for action normalization
def normalize_action(action_space_limits, action):
    """Normalize the action from [low, high] to [-1, 1]"""
    low, high = action_space_limits
    return 2.0 * ((action - low) / (high - low)) - 1.0

def denormalize_action(action_space_limits, action):
    """Denormalize the action from [-1, 1] to [low, high]"""
    low, high = action_space_limits
    return low + (0.5 * (action + 1.0) * (high - low))

# Wrapper for easy and uniform interfacing with SB3
class GymDssatWrapper(gym.Wrapper):
    def __init__(self, env):
        super(GymDssatWrapper, self).__init__(env)

        self.action_low, self.action_high = self._get_action_space_bounds()

        # using a normalized action space
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(1,), dtype="float32")

        # using a vector representation of observations to allow
        # easily using SB3 MlpPolicy
        self.observation_space = gym.spaces.Box(low=0.0,
                                                high=np.inf,
                                                shape=env.observation_dict_to_array(
                                                    env.observation).shape,
                                                dtype="float32"
                                                )
        
        # to avoid annoying problem with Monitor when episodes end and things are None
        self.last_info = {}
        self.last_obs = None

    def _get_action_space_bounds(self):
        box = self.env.action_space['anfer']
        return box.low, box.high

    def _format_action(self, action):
        return { 'anfer': action[0] }

    def _format_observation(self, observation):
        return self.env.observation_dict_to_array(observation)

    def reset(self):
        return self._format_observation(self.env.reset())
        

    def step(self, action):
        # Rescale action from [-1, 1] to original action space interval
        denormalized_action = denormalize_action((self.action_low, self.action_high), action)
        formatted_action = self._format_action(denormalized_action)
        obs, reward, done, info = self.env.step(formatted_action)

        # handle `None`s in obs, reward, and info on done step
        if done:
            obs, reward, info = self.last_obs, 0, self.last_info
        else:
            self.last_obs = obs
            self.last_info = info

        formatted_observation = self._format_observation(obs)
        return formatted_observation, reward, done, info

    def close(self):
        return self.env.close()

    def eval(self):
        return self.env.set_evaluation()

    def __del__(self):
        self.close()

Next, we will create the environment and train the PPO agent...

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor

# Create environment
env_args = {
    'run_dssat_location': '/opt/dssat_pdi/run_dssat',
    'mode': 'fertilization',
    'seed': 123,
    'random_weather': True,
}

env = Monitor(GymDssatWrapper(gym.make('GymDssatPdi-v0', **env_args)))

# Training arguments for PPO agent
ppo_args = {
    'seed': 123, # seed training for reproducibility
}

# Create the agent
ppo_agent = PPO('MlpPolicy', env, **ppo_args)

# Train for 10k timesteps
print('Training PPO agent...')
ppo_agent.learn(total_timesteps=1_000_000)
print('Training done')

Now, we will compare to the baseline agents and display the results....

In [ ]:
# Baseline agents for comparison
class NullAgent:
    """
    Agent always choosing to do no fertilization
    """
    def __init__(self, env):
        self.env = env

    def predict(self, obs, state=None, episode_start=None, deterministic=None):
        action = normalize_action((self.env.action_low, self.env.action_high), [0])
        return np.array([action], dtype=np.float32), obs


class ExpertAgent:
    """
    Simple agent using policy of choosing fertilization amount based on days after planting
    """
    fertilization_dic = {
        40: 27,
        45: 35,
        80: 54,
    }

    def __init__(self, env, normalize_action=False, fertilization_dic=None):
        self.env = env
        self.normalize_action = normalize_action

    def _policy(self, obs):
        dap = int(obs[0][0])
        return [self.fertilization_dic[dap] if dap in self.fertilization_dic else 0]

    def predict(self, obs, state=None, episode_start=None, deterministic=None):
        action = self._policy(obs)
        action = normalize_action((self.env.action_low, self.env.action_high), action)

        return np.array([action], dtype=np.float32), obs

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

#evaluation and plotting functions
def evaluate(agent, n_episodes=10):
    # Create eval env
    eval_args = {
        'run_dssat_location': '/opt/dssat_pdi/run_dssat',
        'mode': 'fertilization',
        'seed': 456,
        'random_weather': True,
    }
    env = Monitor(GymDssatWrapper(gym.make('GymDssatPdi-v0', **eval_args)))

    env.eval()

    returns, _ = evaluate_policy(
        agent, env, n_eval_episodes=n_episodes, return_episode_rewards=True)
    
    env.close()

    return returns

# evaluate agents
null_agent = NullAgent(env)
print('Evaluating Null agent...')
null_returns = evaluate(null_agent)
print('Done')
print('Evaluating PPO agent...')
ppo_returns = evaluate(ppo_agent)
print('Done')

expert_agent = ExpertAgent(env)
print('Evaluating Expert agent...')
expert_returns = evaluate(expert_agent)
print('Done')

# write results to a file to be loaded for display
data = [('null', null_returns), ('ppo', ppo_returns), ('expert', expert_returns)]

In [ ]:
import pickle
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

def plot_results(data):
    data_dict = {}
    for label, returns in data:
        data_dict[label] = returns
    df = pd.DataFrame(data_dict)
    
    ax = sns.boxplot(data=df)
    ax.set_xlabel("policy")
    ax.set_ylabel("evaluation output")
    plt.show()

In [ ]:
plot_results(data)

All done! Go ahead and edit and re-run the code cells above to observe results

## Bonus: Saving best performing model

In this section, we will use a feature from stable-baselines3 known as `callbacks`. They enable different kind of operations during the training of a model. For our purpose here, we will use the `EvalCallback` to periodically evaluate and save the best performing model we find during training.

For more on callbacks in stable-baselines3 see [here](https://stable-baselines3.readthedocs.io/en/master/guide/callbacks.html)

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback

env = Monitor(GymDssatWrapper(gym.make('GymDssatPdi-v0', **env_args)))


# Create the agent
ppo_agent = PPO('MlpPolicy', env, **ppo_args)

# path to save best model found
path = 'ppo'

# eval callback
eval_freq = 1000
eval_env_args = {**env_args, 'seed': 345}
eval_env = Monitor(GymDssatWrapper(gym.make('GymDssatPdi-v0', **eval_env_args)))
eval_callback = EvalCallback(eval_env,
                                eval_freq=eval_freq,
                                best_model_save_path=f'./{path}',
                                deterministic=True)


# Train
print('Training PPO agent...')
ppo_agent.learn(total_timesteps=1_000_000, callback=eval_callback)
ppo_agent.save(f'./{path}/final_model')
print('Training done')

In [ ]:
# evaluate agents
null_agent = NullAgent(env)
print('Evaluating Null agent...')
null_returns = evaluate(null_agent)
print('Done')

print('Evaluating PPO agents...')
ppo_best = PPO.load(f'./{path}/best_model')
ppo_best_returns = evaluate(ppo_best)
ppo_returns = evaluate(ppo_agent)
print('Done')

expert_agent = ExpertAgent(env)
print('Evaluating Expert agent...')
expert_returns = evaluate(expert_agent)
print('Done')


results = [('null', null_returns), ('ppo(best)', ppo_best_returns),  ('ppo(final)', ppo_returns), ('expert', expert_returns)]

In [ ]:
plot_results(results)